# *Internet of Things* (IoT)
## Repaso de operación con Arduino

### David Rozo Osorio, I.M, M.Sc.

## Características Técnicas

- Microcontrolador: ATmega328P
- Pines
  - Built-in LED pin: 13
  - Digital I/O pines: 14
  - Analog input pines: 6
  - PWM pines (~): 6
- Comunicación
  - UART: Si
  - I2C: Si
  - SPI: Si
- Alimentación:
  - I/O Voltaje: 5V
  - Voltaje entrada (nominal): (7-12) V
  - Corriente DC, por I/O Pin: 20 mA
  - Conector tipo Barrel
- Velocidad reloj
  - Procesador principal: ATmega328P 16 MHz
  - Procesador USB-Serial: ATmega16U2 16 MHz
- Memoria: ATmega328P	2KB SRAM, 32KB FLASH, 1KB EEPROM
- Dimensions
  - Peso: 25 g
  - Ancho: 53.4 mm
  - Largo: 68.6 mm
- Pines de interrupción: 2, 3

## Pinout
<img src="Imag/arduino_pinout.png" width=800 height=600 />

## Ejemplo 1 - Blink

```cpp
// the setup function runs once when you press reset or power the board
void setup() {
  // initialize digital pin LED_BUILTIN as an output.
  pinMode(LED_BUILTIN, OUTPUT);
}

// the loop function runs over and over again forever
void loop() {
  digitalWrite(LED_BUILTIN, HIGH);   // turn the LED on (HIGH is the voltage level)
  delay(1000);                       // wait for a second
  digitalWrite(LED_BUILTIN, LOW);    // turn the LED off by making the voltage LOW
  delay(1000);                       // wait for a second
}
```



## Ejemplo 2 - Sampling

```cpp
unsigned long currentMicros;
unsigned long previousMicros;
unsigned long elapsedMicros;
unsigned long duration = 10; // milliseconds
int count = 0;

// the setup function runs once when you press reset or power the board
void setup() {
  // initialize digital pin LED_BUILTIN as an output.
  pinMode(LED_BUILTIN, OUTPUT);
  Serial.begin(115200);
}

// the loop function runs over and over again forever
void loop() {

  currentMicros = micros();
  elapsedMicros = currentMicros - previousMicros;
  if (elapsedMicros >= duration){
      Serial.println(count); 
      count++;
      previousMicros = currentMicros;
  } // end time pass check
}
```



<img src="Imag/MonsterShield.png" width=400 height=400 />

<img src="Imag/pin_color_name.png" width=600 height=600 />

<img src="Imag/motor.png" width=400 height=400 />

## Ejemplo 3 - Digital and PWM

```cpp
//MOTOR 1
#define BRAKE 0
#define CW    1
#define CCW   2

#define MOTOR_A1_PIN 7
#define MOTOR_B1_PIN 8
#define EN_PIN_1 A0
#define PWM_MOTOR_1 5
#define MOTOR_1 0

short usSpeed = 150;  //default motor speed
unsigned short usMotor_Status = BRAKE;

void setup()                         
{
    pinMode(MOTOR_A1_PIN, OUTPUT);
    pinMode(MOTOR_B1_PIN, OUTPUT);
    pinMode(PWM_MOTOR_1, OUTPUT);
    pinMode(EN_PIN_1, OUTPUT);
    
    Serial.begin(9600);              // Initiates the serial to do the monitoring 
}    
    
void loop() 
{    
  char user_input;   
 
  
  while(Serial.available())
  {
    user_input = Serial.read(); //Read user input and trigger appropriate function
    digitalWrite(EN_PIN_1, HIGH);
      
  if (user_input =='1')
    {
       Stop();
    }
    else if(user_input =='2')
    {
      Forward();
    }
    else if(user_input =='3')
    {
      Reverse();
    }
    else if(user_input =='+')
    {
      IncreaseSpeed();
    }
    else if(user_input =='-')
    {
      DecreaseSpeed();
    }
    else
    {
      Serial.println("Invalid option entered.");
    }
      
  }

}

void Stop()
{
  Serial.println("Stop");
  usMotor_Status = BRAKE;
  motorGo(MOTOR_1, usMotor_Status, 0);

}

void Forward()
{
  Serial.println("Forward");
  usMotor_Status = CW;
  motorGo(MOTOR_1, usMotor_Status, usSpeed);

}

void Reverse()
{
  Serial.println("Reverse");
  usMotor_Status = CCW;
  motorGo(MOTOR_1, usMotor_Status, usSpeed);

}

void IncreaseSpeed()
{
  usSpeed = usSpeed + 10;
  if(usSpeed > 255)
  {
    usSpeed = 255;  
  }
  
  Serial.print("Speed +: ");
  Serial.println(usSpeed);

  motorGo(MOTOR_1, usMotor_Status, usSpeed);
}

void DecreaseSpeed()
{
  usSpeed = usSpeed - 10;
  if(usSpeed < 0)
  {
    usSpeed = 0;  
  }
  
  Serial.print("Speed -: ");
  Serial.println(usSpeed);

  motorGo(MOTOR_1, usMotor_Status, usSpeed);
}

void motorGo(uint8_t motor, uint8_t direct, uint8_t pwm)         //Function that controls the variables: motor(0 ou 1), direction (cw ou ccw) e pwm (entra 0 e 255);
{
    if(direct == CW)
    {
      digitalWrite(MOTOR_A1_PIN, LOW); 
      digitalWrite(MOTOR_B1_PIN, HIGH);
    }
    else if(direct == CCW)
    {
      digitalWrite(MOTOR_A1_PIN, HIGH);
      digitalWrite(MOTOR_B1_PIN, LOW);      
    }
    else
    {
      digitalWrite(MOTOR_A1_PIN, LOW);
      digitalWrite(MOTOR_B1_PIN, LOW);            
    }
    
    analogWrite(PWM_MOTOR_1, pwm); 
}

```


## Ejemplo 4 - Interruptions

```cpp

#include <util/atomic.h> // For the ATOMIC_BLOCK macro

unsigned long currentMicros;
unsigned long previousMicros;
unsigned long elapsedMicros;
unsigned long duration = 10; // milliseconds
int count = 0;


const int enca_m1 = 2;//18; //3; // ENCA  14//A pin -> the interrupt pin D5 --- M1
const int encb_m1 = 3;//17; //11; // ENCB  12//B pin -> the digital pin D6 --- M1
int posi_m1;
#define EN_PIN_1 A0

void readEncoder_m1();
void get_posi();

// the setup function runs once when you press reset or power the board
void setup() {
  // initialize digital pin LED_BUILTIN as an output.
  pinMode(LED_BUILTIN, OUTPUT);
  Serial.begin(115200);
    
    
  pinMode(enca_m1,INPUT);
  pinMode(encb_m1,INPUT);
    
  pinMode(EN_PIN_1, OUTPUT);
  digitalWrite(EN_PIN_1, HIGH);
    
    
  attachInterrupt(digitalPinToInterrupt(enca_m1),readEncoder_m1,RISING);
    
}

// the loop function runs over and over again forever
void loop() {

  currentMicros = micros();
  elapsedMicros = currentMicros - previousMicros;
  if (elapsedMicros >= duration){
      get_posi();
      

      previousMicros = currentMicros;
  } // end time pass check
}

void get_posi()
{
   ATOMIC_BLOCK(ATOMIC_RESTORESTATE) {
       Serial.println(posi_m1);

   }
}

void readEncoder_m1(){
  int b = digitalRead(encb_m1);
  if(b > 0){
    posi_m1++;
  }
  else{
    posi_m1--;
  }
}

```